# Hugging Face Projects

This notebook will be dedicated to using Hugging Face in order to code some interesting projects:

1. Text Summarization  
2. 2
3. 3

All of these will be done through fine tuning of existing baseline models.

We will need a GPU in order to fine tune the models:

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## 1. Text Summarization Project (Seq2Seq)

For this task we are going to use a class of models called *Seq2Seq*.

Seq2Seq models map an input sequence to an output sequence — useful for tasks like translation, summarization, dialogue.
Transformer-based Seq2Seq models (like T5 and BART) replaced older RNN-based ones, achieving much better performance.

### 1.1 Install Dependencies

We need some packages in order to start with our project:

In [15]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [1]:
# disinstall and re-install accelerate for gpu acceleration

!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate  # sometimes colab uses older versions
!pip install transformers accelerate  # now we're sure we're using a new version

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: accelerate 1.7.0
Uninstalling accelerate-1.7.0:
  Successfully uninstalled accelerate-1.7.0
  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)
Using cached accelerate-1.7.0-py3-none-any.whl (362 kB)


In [2]:
# import to test that everything is fine

from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer # For the model we're going to use
from datasets import load_dataset, load_from_disk # For the datasets

# python libraries
import matplotlib.pyplot as plt
import pandas as pd

# tokenization
import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm # just progress bar

import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# let's check the device
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")

Using device: cpu


In [20]:
# Choose our model "checkpoint" (ckpt)
model_ckpt = "google/pegasus-cnn_dailymail" # https://huggingface.co/google/pegasus-cnn_dailymail

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [21]:
# load the model and send it to device
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 1.2 Get the Data

In [7]:
# sometimes i have problems loading if i dont update datasets first...
!pip install --upgrade datasets fsspec

In [24]:
# load the dataset
dataset_samsum = load_dataset("knkarthick/samsum") # https://huggingface.co/datasets/knkarthick/samsum

In [25]:
dataset_samsum  # it's composed of dialogue and summary couples

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

In [26]:
dataset_samsum["train"]["dialogue"][1]

'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great'

In [27]:
dataset_samsum["train"]["summary"][1]

'Olivia and Olivier are voting for liberals in this election. '

In [40]:
samsum_train_df = pd.DataFrame(dataset_samsum['train'])
samsum_test_df = pd.DataFrame(dataset_samsum['test'])

#### 1.2.1: Always inspect Your Data Thoroughly...

In [42]:
# I was getting an error when mapping my dataset, went back and checked the data for NaN values...

print(samsum_train_df.isnull().sum())
print(samsum_test_df.isnull().sum())

id          0
dialogue    1
summary     0
dtype: int64
id          0
dialogue    0
summary     0
dtype: int64


In [44]:
samsum_train_df[samsum_train_df.isnull().any(axis=1)] # bad data here

,id,dialogue,summary
6054,13828807,None,problem with visualization of the content


In [54]:
# filter the dataset to remove it
# Define a filter function
def clean_example(example):
    return (example['dialogue'] is not None and
            example['summary'] is not None)

# Apply the filter to each split
dataset_samsum_clean = dataset_samsum.map(lambda x: x, remove_columns=[])  # make a copy
dataset_samsum_clean['train'] = dataset_samsum['train'].filter(clean_example)
dataset_samsum_clean['validation'] = dataset_samsum['validation'].filter(clean_example)
dataset_samsum_clean['test'] = dataset_samsum['test'].filter(clean_example)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

Filter:   0%|          | 0/819 [00:00<?, ? examples/s]

> **Note:** Hugging Face DatasetDict objects are immutable by default.
>
> When you apply `.filter()`, it returns a new object — it doesn't modify the original
dataset in-place.
>
>If you want to keep your original `dataset_samsum` untouched, you can make a copy before applying filters.
>```python
dataset_samsum_clean = dataset_samsum.map(lambda x: x, remove_columns=[])
```
>This trick is used to make a shallow copy of the dataset before you start modifying (filtering) it, to avoid messing up the original.
>
> In this case we didn't really need to keep the original with NaN values, but just for safety I made a copy first.

In [56]:
samsum_train_df = pd.DataFrame(dataset_samsum_clean['train'])
print(samsum_train_df.isnull().sum())
print(samsum_test_df.isnull().sum())

id          0
dialogue    0
summary     0
dtype: int64
id          0
dialogue    0
summary     0
dtype: int64


### 1.3 Preprocess data (embedding)

In [49]:
def convert_examples_to_features(example_batch):
  """
  Encodes the dataset in batches
  """

  input_encodings = tokenizer(example_batch['dialogue'],
                              padding='max_length',
                              max_length=1024,
                              truncation=True)

  with tokenizer.as_target_tokenizer(): # target tokenizer context manager (see below)
    target_encodings = tokenizer(example_batch['summary'],
                                 padding='max_length',
                                 max_length=128,
                                 truncation=True)

  return {  # tutti i tokenizer ritornano input_ids attention_mask etc.? o Hanno strutture diverse
            'input_ids' : input_encodings['input_ids'],
            'attention_mask' : input_encodings['attention_mask'],
            'labels' : target_encodings['input_ids']
  }

> **Note:**
>
> In sequence-to-sequence (seq2seq) models like Pegasus, it is essential to differentiate between input tokens and target tokens during tokenization. Although the tokenizer might appear the same for both, using `tokenizer.as_target_tokenizer()` ensures that tokenization parameters and settings are properly adjusted for the target side (decoder). This is crucial because the model processes the source text through the encoder and generates the target text through the decoder. Properly tokenizing targets guarantees that the model receives the correct input format for loss computation and sequence generation. Without this distinction, the model could misinterpret the labels, leading to incorrect training and poor performance.


In [57]:
# apply tokenization with map
dataset_samsum_pt = dataset_samsum_clean.map(convert_examples_to_features,
                                             batched=True)


Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

## 1.4 Training

### 1.4.1 Data Collator

When we have a huge amount of data, it's easy for our machine to run out of memory while training if we load all the data at once.

To mitigate this issue, we can use the [`DataCollator`](https://huggingface.co/docs/transformers/main_classes/data_collator#data-collator) class.